## Исследование надёжности заёмщиков

Цель: изучить влияет ли семейное положение и количество детей клиента банка на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

### Вывод

- есть прямая зависимость между наличием детей и возвратом кредита в срок. Чем меньше детей у клиента, тем выше вероятность своевременного возврата кредитных средств. 
- наблюдается зависимость между семейным положением и возвратом кредита в срок. Клиенты из категории "не женат / не замужем" хуже всего возращают кредиты, а люди состоящие в браке наоборот. 
- зависимости между уровнем дохода и возвратом кредита в срок не выявлено. 
- вероятность задолженности по кредиту на недвижимость минимальна. Хуже всего возращают кредитные средства, взятые на образование и автомобиль. 


### Ключевые понятия

Pandas, PyMystem3, Python, лемматизация, предобработка данных

### План выполнения проекта


1. [Изучим общую информацию о данных](#step1)


2. [Предобработка данных](#step2)


- определим и заполним пропущенные значения;
- удалим дубликаты;
- выделим леммы в значениях столбца с целями получения кредита;
- категоризируем данные.


3. [Ответим на вопросы:](#step3)


- есть ли зависимость между наличием детей и возвратом кредита в срок?
- есть ли зависимость между семейным положением и возвратом кредита в срок?
- есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- как разные цели кредита влияют на его возврат в срок?


4. [Напишем общий вывод](#step4)

### Описание датасета

**В таблице имеется 12 следующих столбцов:**

1. children — количество детей в семье
2. days_employed — общий трудовой стаж в днях
3. dob_years — возраст клиента в годах
4. education — уровень образования клиента
5. education_id — идентификатор уровня образования
6. family_status — семейное положение
7. family_status_id — идентификатор семейного положения
8. gender — пол клиента
9. income_type — тип занятости
10. debt — имел ли задолженность по возврату кредитов
11. total_income — ежемесячный доход
12. purpose — цель получения кредита

### Откроем файл и изучим общую информацию <a id="step1"></a>   

In [31]:
# импортируем необходимые для работы библиотеки
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [32]:
data_clients = pd.read_csv('/home/ilya/Desktop/Datasets/data_bank_clients.csv')

In [33]:
data_clients.info()
display(data_clients.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В таблице 21525 строк и 12 столбцов. Тип данных в таблице: float64(2 столбца), int64(5 столбцов), object(5 столбцов). В столбцах 'days_employed', 'total_income' меньше строк, чем во всех остальных - 19351, что свидетельствует о пропусках. Пропуски могли возникнуть по причине ожидания банком информации от клиентов, либо эти пропуски допущены сознательно, ввиду того, что информация устарела.

В столбце 'days_employed' есть отрицательные значения. Более того в 4-й строке стаж соответствует почти 100 годам. Это значит что данные не соответствуют реальности. Возможно при выгрузке данных произошла ошибка. Необходимо обратиться к разработчикам.

In [34]:
data_clients.describe()
# с помощью метода describe() посмотрим какие отклонения есть в данных

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Видно, что в столбце "children" есть значение "-1" и "20". В первом случае заменим значение на положительное. Во втором случае очевидно что ноль напечатан ошибочно. Заменим "20" на "2". Используем метод replace.

In [35]:
data_clients ['children']=data_clients['children'].replace(-1, 1)
data_clients ['children']=data_clients['children'].replace(20, 2)
data_clients.describe()
# с помощью метода describe() удостоверимся в успешной замене данных

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.479721,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,0.755528,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Предобработка данных <a id="step2"></a>   
#### Обработка пропусков

In [36]:
print(data_clients.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Количество пропущенных строк в столбцах 'days_employed' и 'total_income' одинаково. Это свидетельствует о том, что пропуски допущены в одних и тех же местах. Проверим данное предположение.

In [37]:
len(data_clients[(data_clients['days_employed'].isna()==True) & (data_clients['total_income'].isna()==True)])

2174

Пропущенные значения в столбце 'days_employed' заменим на нули, так как данные изначально некорректны (для этого применим метод fillna()). На этом оставим столбец с трудовым стажем в покое. Кредитный отдел банка не интересует какое влияние на возврат кредитных средств оказывает стаж заёмщика.

In [38]:
data_clients['days_employed'] = data_clients['days_employed'].fillna(value=0)
# заменили пропуски в столбце на нули
data_clients.info()
# вызовом метода info() убедимся, что замена произошла успешно

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из данных мы видим, что среди заёмщиков всего 2 безработных, 1 в декрете и 1 студент. Это подтверждает предположение, что пропуски в таблице допущены не по причине отсутствия дохода.
На уровень дохода прямым образом влияет тип занятости. Очевидно, что студент имеет меньший уровень дохода нежели сотрудник компании. Проведем группировку по столбцу income_type и вычислим доходную медиану для каждого типа занятости, отсортируем по возрастанию.

In [39]:
data_median_total_income = data_clients.groupby('income_type')['total_income'].median().sort_values()
print(data_median_total_income)

income_type
в декрете           53829.130729
студент             98201.625314
пенсионер          118514.486412
безработный        131339.751676
сотрудник          142594.396847
госслужащий        150447.935283
компаньон          172357.950966
предприниматель    499163.144947
Name: total_income, dtype: float64


Вместо пропусков присвоим каждому типу занятости соответствующую медиану.

In [40]:
#Создаем функцию, которая принимает датасет и тип_занятости, после чего заменяет пустые значения на медианные
def fill_by_median(data_clients, job_type):
    data_clients.loc[data_clients['income_type'] == job_type,'total_income'] = data_clients\
    .loc[data_clients['income_type'] == 
job_type,'total_income'].fillna(data_median_total_income[job_type])
    
# Применяем функцию
fill_by_median(data_clients, 'сотрудник')

fill_by_median(data_clients, 'компаньон')

fill_by_median(data_clients, 'пенсионер')

fill_by_median(data_clients, 'госслужащий')

fill_by_median(data_clients, 'предприниматель')
data_clients.info()
# вызовом метода info() убедимся, что замена произошла успешно

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [41]:
#после преобразований проверим количество пропущенных строк в столбцах 'days_employed' и 'total_income'
print(data_clients.isnull().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод по данному пункту:** пропуски были в 2-х столбцах: 'days_employed' и 'total_income'. В первом столбце отсутсвующие значения заменили на нули (в запросе банка данный столбец не коррелирует с фактом погашения кредита в срок). Помимо этого, очевидно, что данные всего столбца некорректны, о чём было написано выше. Для столбца 'total_income' высчитали доходную медиану по каждому типу занятости и заменили ею пропущенные значения. Для доходов приоритетен метод median() нежели mean(). Это связано с тем, что заработок может очень сильно разнитьса и среднее значение использовать некорректно.

#### Замена типа данных

In [42]:
#заменим вещественный тип данных на целые числа применив метод .astype() с аргументом int
data_clients['days_employed'] = data_clients['days_employed'].astype(int)
data_clients['total_income'] = data_clients['total_income'].astype(int)
#с помощью метода info() проверим тип данных в таблице после изменений
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод по данному пункту:** в 2-х столбцах:'days_employed' и 'total_income' был вещественный тип данных (float64). Рабочий стаж предполагает целые числа, а десятичными данными в доходе можно пренебречь и улучшить восприятие чисел. Для перевода вещественных данных в целые числа применили метод astype() перевели стаж и доход в целые числа. Теперь данные в таблице приведены к общему типу int64 (7 столбцов) и object (5 столбцов).

#### Обработка дубликатов

С помощью метода duplicated() посмотрим количество дубликатов строк в таблице.

In [43]:
data_clients.duplicated().sum()

54

Всего 54 строки полностью дублируются. Удалим их с помощью метода drop_duplicates.

In [44]:
data_clients = data_clients.drop_duplicates().reset_index(drop = True)

In [45]:
data_clients.duplicated().sum()
# проверим успешно ли прошло удаление дубликатов

0

Теперь найдём дубликаты с учётом региста. Искать их будем только в столбцах с типом данных object.

In [46]:
data_clients['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Мы видим, что каждый тип образования имеет 3 варианта написания. Будем применять метод str.lower(), это позволит привести все варианты написания к нижнему регистру.

In [47]:
data_clients['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

В этом столбце всё записано нижним регистром. Изменений не требуется.

In [48]:
data_clients['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

В этом столбце всё записано нижним регистром. Изменений не требуется.

In [49]:
data_clients['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В этом столбце множество целей, на которые берётся кредит. Нужно будет выделить основные кредитные цели и применить лемматизацию.

Обрабатываем дубликаты только в одном столбце - 'education' с учётом регистра.

In [50]:
data_clients['education_lowercase'] = data_clients['education'].str.lower() 
#переведём виды образования в нижний регистр методом str.lower(). Сохраним их в новом столбце 'education_lowercase'.
#в столбце 'education_lowercase' посчитаем количество дублей для каждого вида образования методом value_counts()
data_clients['education_lowercase'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education_lowercase, dtype: int64

**Вывод по данному пункту:** 54 строки таблицы являлись дубликатами, данные дубликаты были удалены. Дубликаты с учетом регистра присутствовали столбце 'education'. Применили ручной поиск дубликатов с учетом регистра. Привели всё к нижнему регистру методом str.lower(). Дубликаты в видах образования могли появиться в результате ручного ввода данных или выгрузки данных с разных баз данных, а каждой из них, в свою очередь, применялся свой вариант написания.

#### Лемматизация

In [51]:
m = Mystem()
def purpose_lemmas(purpose):
    lemma = ' '.join(m.lemmatize(purpose))
    return lemma
data_clients['purpose_lemmas'] = data_clients['purpose'].apply(purpose_lemmas)
# используя метод apply() добавили новый столбец 'purpose_lemmas'. 
data_clients['purpose_lemmas'].value_counts()
# посмотрим на результаты столбца purpose_lemmas

автомобиль \n                                    972
свадьба \n                                       793
на   проведение   свадьба \n                     773
сыграть   свадьба \n                             769
операция   с   недвижимость \n                   675
покупка   коммерческий   недвижимость \n         662
покупка   жилье   для   сдача \n                 652
операция   с   жилье \n                          652
операция   с   коммерческий   недвижимость \n    650
покупка   жилье \n                               646
жилье \n                                         646
покупка   жилье   для   семья \n                 638
строительство   собственный   недвижимость \n    635
недвижимость \n                                  633
операция   со   свой   недвижимость \n           627
строительство   жилой   недвижимость \n          625
покупка   недвижимость \n                        621
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

Условно все цели кредита можно разделить на 4 категории: свадьба, недвижимость, автомобиль и образование. Цель кредита "ремонт" можно отнести к недвижимсоти, так как это операция связанна непосредственно с ней. Все операции с покупкой коммерческой и некоммерческой недвижимости, а также строительством отнесем к категории недвижимости, так как суть этих целей одна. 

In [52]:
# Напишем цикл, который будет проходить по этому столбцу и в другой столбец записывать результат

def purpose_category(column):
    if 'ремонт' in column:
        return 'недвижимость'
    if 'жилье' in column:
        return 'недвижимость'
    if 'недвижимость' in column:
        return 'недвижимость'
    if 'автомобиль' in column:
        return 'автомобиль'
    if 'образование' in column:
        return 'образование'
    if 'свадьба' in column:
        return 'свадьба'
data_clients['purpose_category'] = data_clients['purpose_lemmas'].apply(purpose_category)
data_clients['purpose_category'].value_counts()
display(data_clients)
# проверим добавление столбца purpose_category

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_lemmas,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высшее,покупка жилье \n,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,среднее,приобретение автомобиль \n,автомобиль
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,среднее,покупка жилье \n,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,среднее,дополнительный образование \n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,среднее,сыграть свадьба \n,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,среднее,операция с жилье \n,недвижимость
21467,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,среднее,сделка с автомобиль \n,автомобиль
21468,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,среднее,недвижимость \n,недвижимость
21469,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,среднее,на покупка свой автомобиль \n,автомобиль


**Вывод по данному пункту:** в процессе лемматизации выделили 4 кредитных целей. Создали столбец "purpose_category", в который записали цель каждого займа в соответствии с выделенными категориями.

#### Категоризация данных

При просмотре всех столбцов таблицы можно заметить что образование и семейное положение встречаются дважды: family_status и family_status_id, education и education_id. Это не только увиличивает таблицу визуально, но и утяжеляет её, увеличивает время обработки данных в таблице.

Целесообразно создать "словари", в которых будут собраны все варианты образование/ семейное положение и присвоенные им ID.

In [53]:
education_dict = data_clients[['education_lowercase','education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict.sort_values('education_id')

,education_lowercase,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [54]:
family_status_dict = data_clients[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict.sort_values('family_status_id')

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Проведем категоризацию данных по уровню дохода. Разделим клиетов банка на 4 группы применив метод разбиения по квартилям qcut().

In [55]:
pd.qcut(data_clients['total_income'], 4)

0        (195767.5, 2265604.0]
1         (107654.5, 142594.0]
2         (142594.0, 195767.5]
3        (195767.5, 2265604.0]
4         (142594.0, 195767.5]
                 ...          
21466    (195767.5, 2265604.0]
21467     (142594.0, 195767.5]
21468    (20666.999, 107654.5]
21469    (195767.5, 2265604.0]
21470    (20666.999, 107654.5]
Name: total_income, Length: 21471, dtype: category
Categories (4, interval[float64]): [(20666.999, 107654.5] < (107654.5, 142594.0] < (142594.0, 195767.5] < (195767.5, 2265604.0]]

**Вывод по данному пункту:** исходя из полученных интервалов определим низкий доход - не более 107654.5, средний доход до 142594, высокй уровень дохода до 195767.5, всё что больше - высокий уровень дохода.

### Ответим на вопросы <a id="step3"></a>   

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [56]:
# для ответа на этот вопрос создадим сводную таблицу
child_pivod = data_clients.pivot_table(index=['children'], values='debt', aggfunc = ['count', 'mean'])
# присвоим название новым столбцам
child_pivod.set_axis(['количество заёмщиков', 'вероятность задолженности'], axis='columns', inplace=True)
# отсортируем сводную таблицу по вероятности задолженности
child_pivod.sort_values(by = 'вероятность задолженности')

,количество заёмщиков,вероятность задолженности
children,,
5,9,0.000000
0,14107,0.075353
3,330,0.081818
1,4856,0.091639
2,2128,0.094925
4,41,0.097561


**Вывод по данному пункту:** мы можем пренебречь статистикой заёмщиков, которые имеют 3-х, 4-х и 5 детей, так как их процент ничтожно мал (менее 2% от общего числа заёмщиков). Соответственно, рассмотрим людей, не имеющих детей, а также заёмщиков, у которых 1 и 2-ое детей.
- у заёмщика нет детей - вероятность задоженности 7,5%
- у заёмщика 1 ребёнок - вероятность задоженности 9,2%
- у заёмщика 2 детей - вероятность задоженности 9,4%.
Исходя из этого мы можем сделать вывод, что наличие детей негативно влияет на отдачу кредита. Более того, с увеличением количества детей, возрастает вероятность формирования долгов по кредитам. Бездетные клиенты банка являются более надёжными.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [57]:
# для ответа на этот вопрос создадим сводную таблицу
family_pivod = data_clients.pivot_table(index=['family_status'], values='debt', aggfunc = ['count', 'mean'])
# присвоим название новым столбцам
family_pivod.set_axis(['количество заёмщиков', 'вероятность задолженности'], axis='columns', inplace=True)
# отсортируем сводную таблицу по вероятности задолженности
family_pivod.sort_values(by = 'вероятность задолженности')

,количество заёмщиков,вероятность задолженности
family_status,,
вдовец / вдова,959,0.065693
в разводе,1195,0.071130
женат / замужем,12344,0.075421
гражданский брак,4163,0.093202
Не женат / не замужем,2810,0.097509


**Вывод по данному пункту:** Здесь никакими данными мы пренебречь не можем. Самая малочисленная категория "вдова / вдовец" составляет более 4% от общего числа заёмщиков.
Исходя из данных можно сделать вывод, что самой многочисленной, и в то же время одной из самых платёжеспособных категорий, являются заёмщики состоящие в браке (вероятность задолженности 7,5%). Также хорошие показатели платёжеспособности у вдовцов и разведенных клиентов (6,6% и 7,1% соответственно). А вот люди, живущие гражданским браком и одинокие имеют худшие показатели (вероятность задолженности 9,3% и 9,7% соответственно).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для того, чтобы выяснить зависимость между уровнем дохода и возвратом кредита в срок, необходимо заёмщиков сгруппировать по уровню доходов. 

In [58]:
def income_category(x):
    if x < 107654.5:
        return 'low_inc'
    if 107654.5 < x < 142594.0:
        return 'middle_inc'
    if 142594.0 < x < 195767.5:
        return 'high_inc'
    else:
        return 'very_high_inc'
    
data_clients['income_category'] = data_clients['total_income'].apply(income_category)
data_clients['income_category'].value_counts()

very_high_inc    6445
low_inc          5368
high_inc         5245
middle_inc       4413
Name: income_category, dtype: int64

**Вывод по данному пункту:** из полученных данных видно, что самой многочисленной категорией являются заёмщики с уровнем дохода более 196 тысяч, а самой малочисленной со средним уровнем дохода. Теперь проверим как уровень дохода влиет на возрат кредита в срок.

In [59]:
# создадим сводную таблицу
income_pivod = data_clients.pivot_table(index=['income_category'], values='debt', aggfunc = ['count', 'mean'])
# присвоим название новым столбцам
income_pivod.set_axis(['количество заёмщиков', 'вероятность задолженности'], axis='columns', inplace=True)
# отсортируем сводную таблицу по вероятности задолженности
income_pivod.sort_values(by = 'вероятность задолженности')

,количество заёмщиков,вероятность задолженности
income_category,,
very_high_inc,6445,0.074631
low_inc,5368,0.079545
high_inc,5245,0.085415
middle_inc,4413,0.087242


Неожиданные выводы: самыми порядочными оказались не только с люди с очень высоким доходом, но и с низким! Вероятность задолженности 7,4% и 7,9% соответственно. 
Самой опасной категорией для банка оказались "середнячки". Вероятность задолженности у них самая высокая и составляет 8,7%. Люди с высоким уровнем дохода также представляют опасность, вероятность задолженности у них не многим ниже чем у предыдущей группы - 8,5%. Исходя из этого, можно сделать вывод, что прямой зависимости между уровнем дохода и возвратом кредитных средств нет.

- Как разные цели кредита влияют на его возврат в срок?

In [60]:
# для ответа на этот вопрос создадим сводную таблицу
purpose_pivod = data_clients.pivot_table(index=['purpose_category'], values='debt', aggfunc = ['count', 'mean'])
# присвоим название новым столбцам
purpose_pivod.set_axis(['количество заёмщиков', 'вероятность задолженности'], axis='columns', inplace=True)
# отсортируем сводную таблицу по вероятности задолженности
purpose_pivod.sort_values(by = 'вероятность задолженности')

,количество заёмщиков,вероятность задолженности
purpose_category,,
недвижимость,10814,0.072314
свадьба,2335,0.079657
образование,4014,0.092177
автомобиль,4308,0.093547


**Вывод по данному пункту:**
1. Лучше всего возвращают кредитные средства, взятые на недвижимость. Около 50% людей берёт кредит на данную цель и весьма неплохо эти деньги возвращает. Можно предположить, что к данному типу кредитованию население относится более серьёзно. В этом случае недвижимость находится в залоге у банка и при возникновении задолженности банк имеет право продать квартиру заёмщика. 
2. Свадьбы - немногочисленная, но надёжная цель. Можем предположить, что в большинстве случаев свадьбы "окупаются" и молодожены имеют возможность досрочно погасить существующий кредит.
3. Хуже дела обстоят с кредитами на образование. Возможно дело в том, что кредиты на образование берут молодые люди. 
4. Самая "плохая" категория - автомобиль, вероятность задолженности 9,3%. Возможно это связано с высоким уровнем ДТП в России. Как правило, даже несмотря на круговую страховку, при таком проишествии заёмщик несёт дополнительные траты на ремонт автомобиля. Также стоит отметить, что приобретая автомобиль заёмщик обрекает себя на дополнительные траты по его содержанию: бензин, стоянка, расходные материалы.

### Общий вывод <a id="step4"></a>   

В первую очередь нужно указать кредитному отдела банка не некорректность данных относительно трудового стажа заёмщиков, "отрицательных детей" и семей с 20-ю детьми. Это позволит исключить эту проблему в будущем.

Исходя из проведенного анализа данных можно сделать вывод, что есть прямая зависимость между наличием детей и возвратом кредита в срок. Чем меньше детей у клиента, тем выше вероятность своевременного возврата кредитных средств. Данный вывод логичен, так как дети влекут за собой существенные расходы. Кредитному отделу банка стоит более детально проверять уровень дохода и кредитную историю семей с детьми.

Также наблюдается зависимость между семейным положением и возвратом кредита в срок. Клиенты из категории "не женат / не замужем" хуже всего возращают кредиты, а люди состоящие в браке наоборот. Вероятно, это связано с тем, что при потере работы одним из супругов, кредитные обязательства на себя берёт его партнер. Одинокие же клиенты не могут рассчитывать на такую помощь.

Зависимость между уровнем дохода и возвратом кредита в срок не выявлено. Это связано с тем, что сумма ежемесячного платежа расчитывается исходя из уровня дохода. Банк не выдаёт заемщикам "непосильные" кредиты. Соответсвенно размер кредита прямопропорционален уровню дохода.

Выявлено, что большинство заёмщиков берёт кредит на недвижимость. Вероятность задолженности по данной категории одна из самых минимальных. Большинство клиентов весьма серьёзно относятся к данному типу кредитования, так как в случае задолженности они могут остаться без жилья. Хуже всего возращают кредитные средства, взятые на образование и автомобиль. Кредитному отделу банка стоит обратить внимание на данные категории и более детально изучать заёмщиков, берущих кредит на эти цели.

Таким образом, самый надёжный заёмщик: "женат / замужем" без детей, цель кредита недвижимость.
А самый ненадёжный - "не женат / не замужем", имеющий 1-го ребёнка и более, цель кредита образование или покупка автомобиля.